# **NFT COLLABOT**

> Summary Value Prop Blog

*NFT CollaBot* is a project that designed by obtaining requirements of NFT ecosystem. NFT marketplace from Tezos ecosystem named objkt.com will be starting point of the project. NFT CollaBot aims to make underrated artists more visible. Also, targets to perform extensive visual-oriented statistics for NFT Creators.

### There are **two main features** of the NFT CollaBot:
> NFT Recommendation by Tag

> Stats for NFT Creators

In [ ]:
# importing libraries for data process
import pandas as pd
import numpy as np

# for visualization of the data
from matplotlib import pyplot as plt

# importing libraries to handle web scraping
import requests
import json

# string operations will be handled during data analysis
import re
import string
from io import StringIO

# also using these libraries
import random
import time

# date-related operations exists on the code
from datetime import *

import math

# for error handling
import contextlib

> ### API DOCUMENTATION

Objktcom has a published API documentation
*https://data.objkt.com/docs/#listing*

Documentation points GraphiQL explore page to try queries.
There is a link below that explains how to write queries with GraphiQL
*https://graphql.org/learn/queries/*

Besides, there is a useful link to code on Python using GraphiQL queries. Check the following link:
*https://towardsdatascience.com/connecting-to-a-graphql-api-using-python-246dda927840*

In [ ]:
# objktcom api endpoint will be used for several times to evaluate queries

api_endpoint = 'https://data.objkt.com/v2/graphql'

According to mail from objkt.com, API Endpoint will move to Version 3 after 30 January 2023.

In [ ]:
def check_API_launch_datetime():
    new_API_launchTime = date(2023,1,20)    # assign the launch time
    current_date=date.today()
    if current_date<new_API_launchTime:     # if earlier than the launch time
        return api_endpoint
    else:
        return 'https://data.objkt.com/v3/graphql'

> ##### GRAPHIQL
GraphiQL playground is useful tool to understand insight of the query format
>  https://data.objkt.com/explore/

> #### Retrieve Beginning Primary Key Value

There are thousands of NFTs belongs to a tag. It will take too long to get all of the data. Besides, it will be more effective to show newer NFTs to achieve main target of the project.

The mechanism have to use token_pk to request data, according to API constrains. Decided to choose a token_pk using a timestamp.

In [ ]:
# query for getting primary key has evaluated below:

#initial_token_pk_query="""query {
#    token(where: {timestamp: {_gt: "2022-06-15T00:12:00+00:00"}}, limit: 1) {
#    creators {
#        token_pk
#    }
#    }
#}"""

"""
# send request
token_pk_data = requests.post(api_endpoint, json={'query': initial_token_pk_query})
token_pk_data = json.loads(token_pk_data.text)
token_pk_data=token_pk_data['data']['token']

# output is 8682680
# this primary key will be used in the next steps of the development
"""

## *Feature 01*: NFT Recommendation Mechanism

> ### RECURSIVE MECHANISM TO GET TOKEN PRIMARY KEY DATA

After getting primary key of the starting point, query for tag of token will be evaluated. In this query, only primary keys of tokens will be requested from API. Then, this primary key value will be used to get information about related NFT. This mechanism aims to implement request process recursively.

In [ ]:
counter=[0]      # counts how many times has the function is executed

def request_nft_token_pk(tag_name_variable):
      # sourcery skip: remove-unnecessary-else, swap-if-else-branches
      if counter[0]>0:global pk_value
      if counter[0]==0: pk_value=8682680

      # declare query to request data from api
      nftToken_pk_query="""query {
      token_tag(where: {tag: {name: {_eq: tag_name_variable}}, _and: {token_pk: {_gt: last_pk_value}}}) {
      token_pk
      }
      }"""
      #create dynamic query to request recursively
      nftToken_pk_query=nftToken_pk_query.replace("tag_name_variable",tag_name_variable)
      nftToken_pk_query=nftToken_pk_query.replace("last_pk_value",str(pk_value))

      #request data from objkt.com api endpoint
      token_pk_request = requests.post(api_endpoint, json={'query': nftToken_pk_query})
      token_pk_request = json.loads(token_pk_request.text)
      token_pk_request=token_pk_request['data']['token_tag']

      #have to assign a boolean variable to check is response empty or not
      isEmpty_response=False

      if counter[0]>0:                       # to avoid unbound local error...
            global primaryKey_df             # ...declare as global
            global primaryKey_df_loop

      if counter[0]==0 :
            primaryKey_df = pd.DataFrame()
            primaryKey_df_loop = pd.DataFrame()
      primaryKey_df_loop = pd.DataFrame(token_pk_request)            # convert json data into a data frame
      primaryKey_df=pd.concat([primaryKey_df,primaryKey_df_loop])    # concatenate two data frames to save all data in one data frame


      if primaryKey_df_loop.shape[0]>0:         # assign latest primary key value to a new variable to use this value for recursion queries
            pk_value=int(primaryKey_df_loop.loc[(primaryKey_df_loop.shape[0])-1].values)
      else: isEmpty_response=True               # make boolean value True whenever response is empty

      # increase one the variable for each execution of recursive function
      counter[0]+=1

      # create recursive algorithm inside function
      if not isEmpty_response:     # execute the function since it does not retrieve any data from api endpoint
            #time.sleep(0.1)
            return request_nft_token_pk(tag_name_variable)
      else:
            primaryKey_df.set_index(np.arange(primaryKey_df.shape[0]))
            return primaryKey_df

#request_nft_token_pk("cartoon")

# note!: arrange index of the data frame

> ### How NFT Recommendation Mechanism Works?

>> **Random Recommendation**

Select an NFT randomly to perform functionality more fair for everyone

>> Bonus: **Future Aspects:**

On the other hand, this mechanism will be extended with an algorithm in further stages

There will be an input option to ask for only one specific marketplace *[e.g. recommand hic et nunc only]*

In [ ]:
import random
randomNFT_index=random.sample(range(primaryKey_df.shape[0]),10)
randomNFT=primaryKey_df.iloc[randomNFT_index[0]]

print(randomNFT['token_pk'])

> ### Get NFT Info From API

One NFT is chosen by random module. This NFT's info will be requested from API.

In [ ]:
#queries must be evaluated for the chosen primary key

nft_info_query= """query{
  token(where: {pk: {_eq: "random_pk_value"}}) {
    name
    token_id
    mime
    supply
    artifact_uri
  }
}
"""

nft_link_query="""query{
  token(where: {pk: {_eq: "random_pk_value"}}) {
    fa {
      token_link
    }
  }
}
"""

nft_active_sale_query="""query{
  token(where: {pk: {_eq: "random_pk_value"}}) {
  	asks(where: {status: {_neq: "cancelled"}}) {
      price
      seller_address
      amount_left
      status
      timestamp
		}
  }
}"""

nft_creator_query="""query{
  token_creator(where: {token_pk: {_eq: "random_pk_value"}}) {
    creator_address
  }
}"""


# quick note...
# "tz1burnburnburnburnburnburnburjAYjjX" is address of burn address for the burned tokens

nft_collectors_query="""query{
  token(where: {pk: {_eq: "random_pk_value"}}) {
    holders{
      quantity
      holder_address
    }
  }
}"""

#replace variable with the primary key value
nft_info_query=nft_info_query.replace("random_pk_value",str(randomNFT['token_pk']))
nft_link_query=nft_link_query.replace("random_pk_value",str(randomNFT['token_pk']))
nft_active_sale_query=nft_active_sale_query.replace("random_pk_value",str(randomNFT['token_pk']))
nft_creator_query=nft_creator_query.replace("random_pk_value",str(randomNFT['token_pk']))
nft_collectors_query=nft_collectors_query.replace("random_pk_value",str(randomNFT['token_pk']))

In [ ]:
nft_info_request = requests.post(api_endpoint, json={'query': nft_info_query})
nft_info_request = json.loads(nft_info_request.text)
nft_info_request = nft_info_request['data']['token']

nft_link_request = requests.post(api_endpoint, json={'query': nft_link_query})
nft_link_request = json.loads(nft_link_request.text)
nft_link_request = nft_link_request['data']['token']

nft_active_sale_request = requests.post(api_endpoint, json={'query': nft_active_sale_query})
nft_active_sale_request = json.loads(nft_active_sale_request.text)
nft_active_sale_request = nft_active_sale_request['data']['token']

nft_creator_request = requests.post(api_endpoint, json={'query': nft_creator_query})
nft_creator_request = json.loads(nft_creator_request.text)
nft_creator_request = nft_creator_request['data']

nft_collectors_request = requests.post(api_endpoint, json={'query': nft_collectors_query})
nft_collectors_request = json.loads(nft_collectors_request.text)
nft_collectors_request = nft_collectors_request['data']['token']

In [ ]:
nft_info_df=pd.DataFrame(nft_info_request)   # convert into data frame
nft_info_df

> #### Generate NFT Link

In [ ]:
nft_link=nft_link_request[0]['fa']['token_link']                      # parse data
nft_link=nft_link.replace(":id",str(int(nft_info_df['token_id'][0]))) # replace a part of string with the token_id to complete url
nft_link

> #### Generate NFT Primary-Secondary Stats

In [ ]:
nft_active_sale_request=nft_active_sale_request[0]['asks']
nft_active_sale_request_df=pd.DataFrame(nft_active_sale_request)

if nft_active_sale_request_df.empty is True: print("no active sale!")
else: print("Available")

> #### Generate Creator Info

In [ ]:
nft_creator_request=nft_creator_request['token_creator']
nft_creator_request=nft_creator_request[0]['creator_address']
nft_creator_request

>> #### TzKT API

TzKT API has an endpoint to get data of wallets on tezos ecosystem. Endpoint link:
>>>https://api.tzkt.io/

In the following stage, have to match creator address with the tezos ecosystem registered name

In [ ]:
import contextlib
def artist_info(wallet_address_input):
    account_data_url=f"https://api.tzkt.io/v1/accounts/{wallet_address_input}"  # tzkt.io API endpoint
    response = requests.get(account_data_url)                                   # send request to get data

    with contextlib.suppress(KeyError or json.decoder.JSONDecodeError):
        response=response.json()
        print("Owner of the wallet address is "+response['alias'])       # printing username of account owner to see function is whether working or not

    account_metadata_url=f"https://api.tzkt.io/v1/accounts/{wallet_address_input}/metadata"        # tzkt.io API endpoint
    response = requests.get(account_metadata_url)                                                  # send request to get data

    try:
        response=response.json()
        if(len(response['instagram'])):
            print("https://www.instagram.com/"+response['instagram']+"/")
    except json.decoder.JSONDecodeError:pass
    except KeyError:                                 # program gets KeyError when relevant metadata does not available for the user
        print("instagram is null")                   # print to test null case, only for validating the code


    # get twitter username from metadata, if exists on metadata
    try:
        response=response.json()
        if(len(response['twitter'])):
            print("https://www.twitter.com/"+response['twitter'])
    except json.decoder.JSONDecodeError:pass
    except AttributeError:
        if(len(response['twitter'])):
            print("https://www.twitter.com/"+response['twitter'])
    except KeyError:print("Twitter is null")

artist_info(nft_creator_request)

## *Feature 02*: Request Artist Info for Their **Stats** From API


> Find Wallet Address of the User by *Tezos Domain* or *Twitter Address*

To improve user experience, Twitter Bot should enable perform stats feature by accepting Tezos Domain as input or recognizing Twitter Address of the User

>> Find Wallet Address by Twitter *(if exists)*

In [ ]:
def findWalletAddress_byTwitter(twitter_address):
    creator_walletAddress_byTwitter_query="""query{
    tzd_domain(where: {token: {holders: {holder: {twitter: {_eq: "twitter_address"}}}}}) {
        owner
        }
    }"""
    evaluated_twitterAddress = f"https://twitter.com/{str(twitter_address)}"
    creator_walletAddress_byTwitter_query = creator_walletAddress_byTwitter_query.replace("twitter_address",evaluated_twitterAddress)

    creator_twitter = requests.post(api_endpoint, json={'query': creator_walletAddress_byTwitter_query})
    creator_twitter = json.loads(creator_twitter.text)
    creator_twitter = creator_twitter ['data']['tzd_domain']
    if creator_twitter != [] and creator_twitter[0]['owner'] != "null":
        return str(creator_twitter[0]['owner'])
    else:            # if query does not respond any name, then there is no wallet matched with the related twitter address
        return False

>> Find Wallet Address by Tezos Domain *(if exists)*

In [ ]:
def findWalletAddress_byTezDomain(tezos_domain):
    creator_walletAddress_byDomain_query="""query findWallet_byDomainAddress {
    tzd_domain(where: {id: {_eq: "tez_domain"}}) {
    owner
        token {
        holders {
            holder {
            twitter
            }
        }
        }
    }
    }"""
    creator_walletAddress_byDomain_query = creator_walletAddress_byDomain_query.replace("tez_domain",tezos_domain)

    creator_tezDomain = requests.post(api_endpoint, json={'query': creator_walletAddress_byDomain_query})
    creator_tezDomain = json.loads(creator_tezDomain.text)
    creator_tezDomain = creator_tezDomain ['data']['tzd_domain']
    if creator_tezDomain != [] and creator_tezDomain[0]['owner'] != "null":
        return str(creator_tezDomain[0]['owner'])
    else:           # if query does not respond any name, then there is no wallet matched with the related tezos domain
        return False

> Checking the Wallet Address is Available or Not

The users may enter wrong inputs, if there is no registered wallet addresses as the input then respond users with an error. Use TzKT API to check input.

In [ ]:
def isWalletAddress(wallet_address):
    account_data_url=f"https://api.tzkt.io/v1/accounts/{wallet_address}" # tzkt.io API endpoint
    response = requests.get(account_data_url)
    with contextlib.suppress(KeyError or json.decoder.JSONDecodeError):
        response=response.json()
        if response['type']=="empty":      # checking the responded data that user exists or not
            return False


def isAvailableWalletAddress(wallet_address):
    if isWalletAddress(wallet_address) is False:
        #print("No tezos wallet address exists with this input")
        return False
    elif wallet_address is False:
        #print("No tezos domain/twitter address exists for this input")
        return False
    else: return True

> Recognizing the Input

The users are enabled to ask their statistics using their tezos wallet addresses, tezos domain or Twitter addresses.

To declare *input format*: The users only enter tezos wallet address/tezos domain or Twitter username. So, script must identify the input.

In [ ]:
def recognize_user_input():
    user_input=str(input("Please enter your tezos wallet/domain or twitter address: "))
    if len(user_input) == 36 and user_input.startswith("tz"):
        return isWalletAddress(user_input)
    elif user_input.endswith(".tez"):
        return findWalletAddress_byTezDomain(user_input)
    elif user_input:
        return findWalletAddress_byTwitter(user_input)
    else: return False

> All NFTs of a Creator

Request token primary keys of a creator's all NFTs to evaluate charts for them

In [ ]:
counter_N=[0]
def creator_allCreated_NFTs(wallet_address):
    if counter_N[0]>0:global nft_pk_val
    if counter_N[0]==0:nft_pk_val=0
    creator_allNFTs_pk_query="""query{
        listing(where: {token: {creators: {creator_address: {_eq: "wallet_address"}, token_pk: {_gt: "nft_pk_val"}}}}, distinct_on: token_pk) {
            token_pk
            timestamp
        }
    }"""
    creator_allNFTs_pk_query = creator_allNFTs_pk_query.replace("nft_pk_val",str(nft_pk_val))
    creator_allNFTs_pk_query = creator_allNFTs_pk_query.replace("wallet_address",str(wallet_address))

    creator_allNFTs_pk = requests.post(api_endpoint, json={'query': creator_allNFTs_pk_query})
    creator_allNFTs_pk = json.loads(creator_allNFTs_pk.text)
    creator_allNFTs_pk = creator_allNFTs_pk['data']['listing']

    # start the mechanism if there are 500 responses
    # otherwise, it is nonsense to wait executing all because one request is enough to get all data
    if len(creator_allNFTs_pk)==500:
        if counter_N[0]>0:
            global creators_allNFTs_pk_df
            global loop_of_allNFT_listings_df
        if counter_N[0]==0:
            creators_allNFTs_pk_df=pd.DataFrame()
            loop_of_allNFT_listings_df=pd.DataFrame()
        loop_of_allNFT_listings_df=pd.DataFrame(creator_allNFTs_pk)
        creators_allNFTs_pk_df=pd.concat([creators_allNFTs_pk_df,loop_of_allNFT_listings_df])
    else:creators_allNFTs_pk_df = pd.DataFrame(creator_allNFTs_pk)

    # there may be multiple listings on primary, so drop duplicates
    creators_allNFTs_pk_df = creators_allNFTs_pk_df.drop_duplicates()
    # convert timestamp attribute data type as date
    creators_allNFTs_pk_df['timestamp']=pd.to_datetime(creators_allNFTs_pk_df['timestamp']).dt.date
    creators_allNFTs_pk_df=creators_allNFTs_pk_df.sort_values(by='timestamp',ascending=True)
    # have to set index again after dropping and sorting operation
    creators_allNFTs_pk_df = creators_allNFTs_pk_df.reset_index()
    del creators_allNFTs_pk_df['index']

    counter_N[0]=+1  # increase counter after each iteration of the function
    if len(creators_allNFTs_pk_df)==500:
        nft_pk_val=str(creators_allNFTs_pk_df['token_pk'][499])
        return creator_allCreated_NFTs(wallet_address)
    else:
        counter_N[0]=0
        return creators_allNFTs_pk_df

> Available Primary & Secondary Listings

There are many productive artists on Tezos ecosystem. Showing their pieces that left on primary would be useful. This functionality will provide them a chance to track their primary and secondary pieces on the market. Additionally, they can see how many of them also sold on secondary at least one time.

**DEVELOPER NOTE**: THIS IS THE LATEST VERSION OF THE RECURSIVE DATA REQUEST MODULE *(23.10.22, 17.44 by GMT+3)*

In [ ]:
def creator_availablePrimary_NFTs(wallet_address):
    if isAvailableWalletAddress(wallet_address) is not True:
        print("You entered unregistered input. Please enter an available wallet address, tezos domain or registered twitter address with your objkt.com profile.")
        return          # to prevent executing rest of the function
    if counter_N[0]>0:global nft_primaryKey_val
    if counter_N[0]==0:nft_primaryKey_val=0
    creator_nft_primaryNFT_info_query="""{
        listing(where: {seller_address: {_eq: "wallet_address"}, status: {_eq: "active"}, token: {creators: {creator_address: {_eq: "wallet_address"}, token_pk: {_gt: "nft_primaryKey_val"}}}}) {
            token_pk
        }
        }
        """
    creator_nft_primaryNFT_info_query = creator_nft_primaryNFT_info_query.replace("nft_primaryKey_val",str(nft_primaryKey_val))
    creator_nft_primaryNFT_info_query = creator_nft_primaryNFT_info_query.replace("wallet_address",str(wallet_address))

    creator_primary_nft_pk = requests.post(api_endpoint, json={'query': creator_nft_primaryNFT_info_query})
    creator_primary_nft_pk = json.loads(creator_primary_nft_pk.text)
    creator_primary_nft_pk = creator_primary_nft_pk['data']['listing']

    # start the mechanism if there are 500 responses
    # otherwise, it is nonsense to wait executing all because one request is enough to get all data
    if len(creator_primary_nft_pk)==500:
        if counter_N[0]>0:
            global creators_availablePrimaryNFTs_pk_df
            global loop_ofPrimary_NFT_listings_df
        if counter_N[0]==0:
            creators_availablePrimaryNFTs_pk_df=pd.DataFrame()
            loop_ofPrimary_NFT_listings_df=pd.DataFrame()
        loop_ofPrimary_NFT_listings_df=pd.DataFrame(creator_primary_nft_pk)
        creators_availablePrimaryNFTs_pk_df=pd.concat([creators_availablePrimaryNFTs_pk_df,loop_ofPrimary_NFT_listings_df])

    else:creators_availablePrimaryNFTs_pk_df = pd.DataFrame(creator_primary_nft_pk)

    # there may be multiple listings on primary, so delete duplicates
    creators_availablePrimaryNFTs_pk_df = creators_availablePrimaryNFTs_pk_df.drop_duplicates()
    creators_availablePrimaryNFTs_pk_df = creators_availablePrimaryNFTs_pk_df.reset_index()     # have to set index again after dropping operation
    del creators_availablePrimaryNFTs_pk_df['index']

    counter_N[0]=+1
    if len(creators_availablePrimaryNFTs_pk_df)==500:
        nft_primaryKey_val=str(creators_availablePrimaryNFTs_pk_df['token_pk'][499])
        return creator_availablePrimary_NFTs(wallet_address)
    else:
        counter_N[0]=0
        return creators_availablePrimaryNFTs_pk_df

> All Sales of an Artist for His/Her Created NFTs

NFT sale history for both primary and secondary of the artists for their created pieces

In [ ]:
counter_N=[0]
def creator_all_NFT_sales(wallet_address):
    if isAvailableWalletAddress(wallet_address) is not True:
        print("You entered unregistered input. Please enter an available wallet address, tezos domain or registered twitter address with your objkt.com profile.")
        return          # to prevent executing rest of the function
    if counter_N[0]>0:global nft_timestamp_val
    if counter_N[0]==0:nft_timestamp_val="2000-01-01T00:00:00+00:00" # initialize the timestamp value
    creator_all_sales_query="""query{
    listing_sale(where: {token: {creators: {creator_address: {_eq: "wallet_address"}}}, timestamp: {_gt: "nft_timestamp_val"}}, distinct_on: timestamp) {
        token_pk
        timestamp
        }
    }"""
    creator_all_sales_query = creator_all_sales_query.replace("nft_timestamp_val",str(nft_timestamp_val))
    creator_all_sales_query = creator_all_sales_query.replace("wallet_address",str(wallet_address))
    creator_all_sales_response= requests.post(api_endpoint, json={'query': creator_all_sales_query})
    creator_all_sales_response = json.loads(creator_all_sales_response.text)
    creator_all_sales_response = creator_all_sales_response['data']['listing_sale']

    if counter_N[0]>0:
        global all_NFT_sales_df
        global loop_NFT_sales_df
    if counter_N[0]==0:
        all_NFT_sales_df=pd.DataFrame()
        loop_NFT_sales_df=pd.DataFrame()

    loop_NFT_sales_df = pd.DataFrame(creator_all_sales_response)
    loop_NFT_sales_df['token_pk']=loop_NFT_sales_df['token_pk'].astype(int)

    all_NFT_sales_df=pd.concat([ all_NFT_sales_df,loop_NFT_sales_df])
    counter_N[0]+=1

    # print(nft_timestamp_val)    # to check how it works

    if len(creator_all_sales_response)==500:  # max retrieves are 500, if less there are no more data to response from api
        nft_timestamp_val=str(loop_NFT_sales_df['timestamp'][499])
        return creator_all_NFT_sales(wallet_address)
    else:
        counter_N[0]=0                     # reset counter in the end
        all_NFT_sales_df=all_NFT_sales_df.reset_index()
        del all_NFT_sales_df['index']      # also reset index, sufficient for the multiple request cases
        return all_NFT_sales_df


> Primary Sales of a Creator

Get Only Primary Sales

In [ ]:
counter_N=[0]
def creator_primary_NFT_sales(wallet_address):
    if counter_N[0]>0:global nft_timestamp_val
    if counter_N[0]==0:nft_timestamp_val="2000-01-01T00:00:00+00:00" # initialize the timestamp value
    creator_primary_sales_query="""{
    listing_sale(where: {token: {creators: {creator_address: {_eq: "wallet_address"}}}, timestamp: {_gt: "nft_timestamp_val"}, seller_address: {_eq: "wallet_address"}}, distinct_on: timestamp) {
        price
        token_pk
        buyer_address
        timestamp
        }
    }"""
    creator_primary_sales_query = creator_primary_sales_query.replace("nft_timestamp_val",str(nft_timestamp_val))
    creator_primary_sales_query = creator_primary_sales_query.replace("wallet_address",str(wallet_address))
    creator_primary_sales_response= requests.post(api_endpoint, json={'query': creator_primary_sales_query})
    creator_primary_sales_response = json.loads(creator_primary_sales_response.text)
    creator_primary_sales_response = creator_primary_sales_response['data']['listing_sale']

    if counter_N[0]>0:
        global all_NFT_sales_df
        global loop_NFT_sales_df
    if counter_N[0]==0:
        all_NFT_sales_df=pd.DataFrame()
        loop_NFT_sales_df=pd.DataFrame()

    loop_NFT_sales_df = pd.DataFrame(creator_primary_sales_response)
    loop_NFT_sales_df['token_pk']=loop_NFT_sales_df['token_pk'].astype(int)

    all_NFT_sales_df=pd.concat([ all_NFT_sales_df,loop_NFT_sales_df])
    counter_N[0]+=1

    if len(creator_primary_sales_response)==500:  # max retrieves are 500, if less there are no more data to response from api
        nft_timestamp_val=str(loop_NFT_sales_df['timestamp'][499])
        return creator_primary_NFT_sales(wallet_address)
    else:
        counter_N[0]=0                # reset counter in the end
        return all_NFT_sales_df

>> Handling Data Frame

* Manipulate Price Value for Exact Amounts
* Handle Timestamp Attribute
* Calculate Income Over Months
* Append Non-existing Months for Accurate Visualization

In [ ]:
# find the first mint date of a creator and return as year-month format
# will be using on multiple functions, creator_primary_sales_df() as well
def find_first_minting_date(wallet_address):
    firstMintDate_ofCreator=creator_allCreated_NFTs(wallet_address)        # assign data frame of all NFTs of the creator
    firstMintDate_ofCreator=firstMintDate_ofCreator.loc[0]['timestamp']    # then assign first NFT's time to the variable
    firstMintDate_ofCreator=firstMintDate_ofCreator.strftime('%Y-%m')      # drop day from the date
    return firstMintDate_ofCreator

# spotting the latest's date in year-month format
def find_last_sale_date(wallet_address):
    last_sale_date=creator_all_NFT_sales(wallet_address)
    last_sale_date=last_sale_date.apply(pd.to_datetime)
    last_sale_date=last_sale_date.loc[len(last_sale_date)-1]['timestamp']
    last_sale_date=last_sale_date.strftime('%Y-%m')
    return last_sale_date

def creator_primary_sales_df(wallet_address):
    creator_primary_sales_dataFrame=creator_primary_NFT_sales(wallet_address)

    # manipulating price column to calculate exact value [as tezos] of a token
    # dividing to 10^6
    creator_primary_sales_dataFrame['price']=pd.to_numeric(creator_primary_sales_dataFrame['price'],downcast="float")
    creator_primary_sales_dataFrame['price']=creator_primary_sales_dataFrame['price']/1000000
    # manipulate timestamp attribute data type as date
    creator_primary_sales_dataFrame['timestamp']=pd.to_datetime(creator_primary_sales_dataFrame['timestamp']).dt.date
    # convert all days to 1 for grouping by year-month pair
    creator_primary_sales_dataFrame['timestamp']=creator_primary_sales_dataFrame['timestamp'].apply(lambda dt: dt.replace(day=1))

    creator_primary_sales_dataFrame = creator_primary_sales_dataFrame.groupby('timestamp').sum()
    del creator_primary_sales_dataFrame['token_pk']

    creator_primary_sales_dataFrame = creator_primary_sales_dataFrame.reset_index()           # convert to data frame from pivot table
    creator_primary_sales_dataFrame['timestamp'] = creator_primary_sales_dataFrame['timestamp'].apply(lambda x: x.strftime('%Y-%m'))
    creator_primary_sales_dataFrame = creator_primary_sales_dataFrame.set_index('timestamp')  # then set date as index

    firstMintDate_ofCreator=find_first_minting_date(wallet_address)
    lastSaleDate_ofCreator=find_last_sale_date(wallet_address)
    def date_range_df(firstMintDate_ofCreator):
        # define a range to fill missing months -if exists- in data frame
        sale_date_range = pd.date_range(
                            start=firstMintDate_ofCreator,         # using the variable for calculating minting range
                            end=lastSaleDate_ofCreator).to_period('m')
        # create a data frame to save all of the months in the range
        sale_date_range=pd.DataFrame(sale_date_range)
        sale_date_range=sale_date_range.drop_duplicates(keep="first")
        sale_date_range['price']= 0
        sale_date_range=sale_date_range.rename(columns={0:'timestamp'})
        sale_date_range['timestamp'] = sale_date_range['timestamp'].apply(lambda x: x.strftime('%Y-%m'))
        sale_date_range=sale_date_range.groupby('timestamp').sum()
        return sale_date_range

    creator_primary_sales=date_range_df(firstMintDate_ofCreator)    # assign the data frame returned from the function

    creator_primary_sales=creator_primary_sales.reset_index()       # then reset index before mapping
    creator_primary_sales_dataFrame=creator_primary_sales_dataFrame.reset_index()

    # use mapping to fill new data frame with values, keep NaN non-existing months on actual data frame
    creator_primary_sales['price']=creator_primary_sales['timestamp'].map(creator_primary_sales_dataFrame.set_index('timestamp')['price'])
    creator_primary_sales=creator_primary_sales.fillna(0)

    return creator_primary_sales.set_index('timestamp')

> Count Sales by Editions Over Months [Only Primary]

Counting number of sales by editions will enable NFT creators to determine amount of editions for next mintings

In [ ]:
def creator_primarySales_byEditions_df(wallet_address):
    creator_primary_sales_dataFrame=creator_primary_NFT_sales(wallet_address)

    # deleting unnecessary attributes from data frame
    del creator_primary_sales_dataFrame['buyer_address']
    del creator_primary_sales_dataFrame['price']

    # manipulate timestamp attribute data type as date
    creator_primary_sales_dataFrame['timestamp']=pd.to_datetime(creator_primary_sales_dataFrame['timestamp']).dt.date
    creator_primary_sales_dataFrame['timestamp']=creator_primary_sales_dataFrame['timestamp'].apply(lambda dt: dt.replace(day=1))
    creator_primary_sales_dataFrame['timestamp']=creator_primary_sales_dataFrame['timestamp'].apply(lambda x: x.strftime('%Y-%m'))

    creator_primary_sales_dataFrame = creator_primary_sales_dataFrame.groupby('timestamp').count()

    # implementing the same algorithm with the function above to fill missing months, in case they exist
    firstMintDate_ofCreator=creator_allCreated_NFTs(wallet_address)
    firstMintDate_ofCreator=firstMintDate_ofCreator.loc[0]['timestamp']
    firstMintDate_ofCreator=firstMintDate_ofCreator.strftime('%Y-%m')

    def date_range_df(firstMintDate_ofCreator):
        sale_date_range = pd.date_range(
                            start=firstMintDate_ofCreator,
                            end=creator_primary_sales_dataFrame.index[len(creator_primary_sales_dataFrame)-1]).to_period('m')
        sale_date_range=pd.DataFrame(sale_date_range)
        sale_date_range=sale_date_range.drop_duplicates(keep="first")
        sale_date_range['token_pk']= 0
        sale_date_range=sale_date_range.rename(columns={0:'timestamp'})
        sale_date_range['timestamp'] = sale_date_range['timestamp'].apply(lambda x: x.strftime('%Y-%m'))
        sale_date_range=sale_date_range.groupby('timestamp').sum()
        return sale_date_range

    creator_primary_sales=date_range_df(firstMintDate_ofCreator)

    creator_primary_sales=creator_primary_sales.reset_index()
    creator_primary_sales_dataFrame=creator_primary_sales_dataFrame.reset_index()

    creator_primary_sales['token_pk']=creator_primary_sales['timestamp'].map(creator_primary_sales_dataFrame.set_index('timestamp')['token_pk'])
    creator_primary_sales=creator_primary_sales.fillna(0)

    creator_primary_sales['token_pk']=creator_primary_sales['token_pk'].astype(int)
    creator_primary_sales=creator_primary_sales.rename(columns={'token_pk':'sold_editions'})

    return creator_primary_sales

>> Visualize Primary Sale Data

In [ ]:
def visualized_creator_primary_sales(wallet_address):
    if isAvailableWalletAddress(wallet_address) is not True:
        print("You entered unregistered input. Please enter an available wallet address, tezos domain or registered twitter address with your objkt.com profile.")
        return         # to prevent executing rest of the function
    dataFrame_toPlot=creator_primary_sales_df(wallet_address)
    # use pandas data frame plot to visualize the data
    dataFrame_toPlot.plot(kind='bar',title=f"Revenue of the Artist ({wallet_address}) on Primary Market by Month",xlabel="Month",ylabel="Revenue (by Tezos)")

> Secondary Sales of the Artist with Royalties Income

* Retrieve Secondary Sales from API
* Get Royalties of Tokens
* Manipulate Timestamp
* Shape with Same Format as Primary Data Frame

There are two functions that gathers secondary sales data frame:

* **creator_secondary_NFT_sales_tokens()** -collects *sale price, token primary key, sale date* and *collector wallet address* from API
* **creator_secondary_NFT_sales_royalties()** -collects *artist royalty* of the sold token from API

In [ ]:
def creator_secondary_NFT_sales_tokens(wallet_address):
    if counter_N[0]>0:global nft_timestamp_val
    if counter_N[0]==0:nft_timestamp_val="2000-01-01T00:00:00+00:00" # initialize the timestamp value
    creator_secondary_sales_query="""{
    listing_sale(where: {token: {creators: {creator_address: {_eq: "wallet_address"}}}, timestamp: {_gt: "nft_timestamp_val"}, seller_address: {_neq: "wallet_address"}}, distinct_on: timestamp) {
        price
        token_pk
        buyer_address
        timestamp
        }
    }"""

    def send_request_sales(query_input):                 # the function is too complicated so wanted to minimize using a function
        query_input = query_input.replace("nft_timestamp_val",str(nft_timestamp_val))
        query_input = query_input.replace("wallet_address",str(wallet_address))

        global response          # avoid UnboundLocal Error
        response = requests.post(api_endpoint, json={'query': query_input})
        response = json.loads(response.text)
        response = response['data']['listing_sale']
        return response

    creator_secondary_sales_response=send_request_sales(creator_secondary_sales_query)

    if counter_N[0]>0:
        global all_secondaryNFT_sales_df
        global loop_secondaryNFT_sales_df
    if counter_N[0]==0:
        all_secondaryNFT_sales_df=pd.DataFrame()
        loop_secondaryNFT_sales_df=pd.DataFrame()

    loop_secondaryNFT_sales_df = pd.DataFrame(creator_secondary_sales_response)
    loop_secondaryNFT_sales_df['token_pk']=loop_secondaryNFT_sales_df['token_pk'].astype(int)
    loop_secondaryNFT_sales_df['price']=loop_secondaryNFT_sales_df['price'].astype(int)
    # loop data frame saves the data for each iteration of the recursive algorithm, it is temporary data source...
    # data frame starts with "all" includes all of the retrieved data, it is permanent data frame that loop data frame transports data
    all_secondaryNFT_sales_df=pd.concat([ all_secondaryNFT_sales_df,loop_secondaryNFT_sales_df])

    counter_N[0]+=1
    if len(creator_secondary_sales_response)==500:  # max retrieves are 500, if less there are no more data to response from api
        nft_timestamp_val=str(loop_secondaryNFT_sales_df['timestamp'][499])
        return creator_secondary_NFT_sales_tokens(wallet_address)
    else:
        counter_N[0]=0
        return all_secondaryNFT_sales_df

In [ ]:
def creator_secondary_NFT_sales_royalties(wallet_address):
    if counter_N[0]>0:global nft_timestamp_val
    if counter_N[0]==0:nft_timestamp_val="2000-01-01T00:00:00+00:00" # initialize the timestamp value
    creator_secondary_sales_royalties_query="""{
    listing_sale(where: {token: {creators: {creator_address: {_eq: "wallet_address"}}}, timestamp: {_gt: "nft_timestamp_val"}, seller_address: {_neq: "wallet_address"}}, distinct_on: timestamp) {
        token {
        royalties {
            amount
            }
        }
    }
    }"""
    def send_request(query_input):                 # the function is too complicated so wanted to minimize using a function
        query_input = query_input.replace("nft_timestamp_val",str(nft_timestamp_val))
        query_input = query_input.replace("wallet_address",str(wallet_address))

        global response          # avoid UnboundLocal Error
        response = requests.post(api_endpoint, json={'query': query_input})
        response = json.loads(response.text)
        response = response['data']['listing_sale']
        return response

    response=send_request(creator_secondary_sales_royalties_query)

    if counter_N[0]>0:
        global all_secondaryNFT_sales_df
        global loop_secondaryNFT_sales_df
    if counter_N[0]==0:
        all_secondaryNFT_sales_df=pd.DataFrame()
        loop_secondaryNFT_sales_df=pd.DataFrame()

    loop_secondaryNFT_sales_df = pd.DataFrame(response)
    # loop data frame saves the data for each iteration of the recursive algorithm, it is temporary data source...
    # data frame starts with "all" includes all of the retrieved data, it is permanent data frame that loop data frame transports data
    all_secondaryNFT_sales_df=pd.concat([ all_secondaryNFT_sales_df,loop_secondaryNFT_sales_df])

    def clean_data(df):
        df['token'] = df['token'].astype(str)
        df['token'] = df['token'].str.replace(r"[a-zA-Z]",'')
        df['token'] = df['token'].str.replace(f'[{string.punctuation}]', '')
        # avoid errors in collaboration cases (in collabs there are multiple royalties. need only 1st)
        df['token'] = [x[:5] for x in df['token']]
        # available to convert numerical data type after necessary operations are implemented
        df['token'] = df['token'].astype(int)
        df['token'] = df['token']/10    # manipulate into exact value
        return df

    clean_data(all_secondaryNFT_sales_df)

    counter_N[0]+=1
    if len(response)==500:  # max retrieves are 500, if less there are no more data to response from api
        nft_timestamp_val=str(loop_secondaryNFT_sales_df['timestamp'][499])
        return creator_secondary_NFT_sales_royalties(wallet_address)
    else:
        counter_N[0]=0
        return all_secondaryNFT_sales_df

In [ ]:
def creator_secondary_NFT_sales(wallet_address):
    royalties_df=creator_secondary_NFT_sales_royalties(wallet_address)
    tokens_df=creator_secondary_NFT_sales_tokens(wallet_address)

    secondary_sales_df = pd.concat([tokens_df,royalties_df], axis=1, join="inner")
    secondary_sales_df['artist_income'] = ""                                        # create a new column to save calculated value
    secondary_sales_df = secondary_sales_df.rename(columns={'token':'royalties'})   # rename to understand purpose of the attribute better
    secondary_sales_df['artist_income'] = (secondary_sales_df[["price", "royalties"]].product(axis=1))
    secondary_sales_df['artist_income'] = secondary_sales_df['artist_income']/100000000

    return secondary_sales_df

In [ ]:
def creator_secondary_sales_df(wallet_address):
    secondary_sales_df=creator_secondary_NFT_sales(wallet_address)
    secondary_sales_df=secondary_sales_df[['timestamp','artist_income']]    # keep only these two columns

    # manipulate timestamp attribute data type as date
    secondary_sales_df['timestamp'] = pd.to_datetime(secondary_sales_df['timestamp']).dt.date
    secondary_sales_df['timestamp'] = secondary_sales_df['timestamp'].apply(lambda dt: dt.replace(day=1))
    secondary_sales_df['timestamp'] = secondary_sales_df['timestamp'].apply(lambda x: x.strftime('%Y-%m'))
    secondary_sales_df = secondary_sales_df.groupby('timestamp').sum()

    firstMintDate_ofCreator=find_first_minting_date(wallet_address)
    lastSaleDate_ofCreator=find_last_sale_date(wallet_address)
    def date_range_df(firstMintDate_ofCreator):
        sale_date_range = pd.date_range(
                            start=firstMintDate_ofCreator,
                            end=lastSaleDate_ofCreator).to_period('m')
        sale_date_range=pd.DataFrame(sale_date_range)
        sale_date_range=sale_date_range.drop_duplicates(keep="first")
        sale_date_range['artist_income']= 0
        sale_date_range=sale_date_range.rename(columns={0:'timestamp'})
        sale_date_range['timestamp'] = sale_date_range['timestamp'].apply(lambda x: x.strftime('%Y-%m'))
        sale_date_range=sale_date_range.groupby('timestamp').sum()
        return sale_date_range
    creator_secondary_sales=date_range_df(firstMintDate_ofCreator)    # assign the data frame returned from the function

    creator_secondary_sales = creator_secondary_sales.reset_index()   # then reset index before mapping
    secondary_sales_df = secondary_sales_df.reset_index()

    # use mapping to fill new data frame with values, keep NaN non-existing months on actual data frame
    creator_secondary_sales['artist_income'] = creator_secondary_sales['timestamp'].map(secondary_sales_df.set_index('timestamp')['artist_income'])
    creator_secondary_sales = creator_secondary_sales.fillna(0)

    return creator_secondary_sales.set_index('timestamp')

> Merge Primary & Secondary Sales Data Frames

Merge two data frames that outputs of these two functions:
* creator_primary_sales_df()
* creator_secondary_sales_df()

In [ ]:
def creator_all_sales_df(wallet_address):
    primary_df = creator_primary_sales_df(wallet_address)
    secondary_df = creator_secondary_sales_df(wallet_address)

    primary_df=primary_df.rename(columns={'price':'primary_income'})
    secondary_df=secondary_df.rename(columns={'artist_income':'secondary_income'})

    return pd.concat([primary_df,secondary_df],axis=1)

Next part is the prepare works for upcoming user story

*Just a sketch notes for a development*

In [ ]:
def creator_secondary_NFT_sales_count(wallet_address):
    secondarySales_df=creator_secondary_NFT_sales(wallet_address)
    secondarySales_df['token_pk']=secondarySales_df['token_pk'].drop_duplicates(keep="first")
    secondarySales_df=secondarySales_df.dropna()
    return len(secondarySales_df)

def numerical_creator_primarySecondary_Stats(wallet_address):
    number_ofAvailablePrimaryNFTs_ofCreator=creator_availablePrimary_NFTs(wallet_address)
    number_ofAvailablePrimaryNFTs_ofCreator=len(number_ofAvailablePrimaryNFTs_ofCreator)
    number_ofSoldSecondaryNFTs_ofCreator=creator_secondary_NFT_sales_count(wallet_address)
    # using the number of all created NFTs of a creator to perform calculations for unsold NFTs on secondary
    number_ofAllCreatedNFTs_ofCreator=creator_allCreated_NFTs(wallet_address)
    number_ofUnsoldSecondaryNFTs_ofCreator=len(number_ofAllCreatedNFTs_ofCreator)-number_ofAvailablePrimaryNFTs_ofCreator-number_ofSoldSecondaryNFTs_ofCreator
    return number_ofAvailablePrimaryNFTs_ofCreator,number_ofSoldSecondaryNFTs_ofCreator,number_ofUnsoldSecondaryNFTs_ofCreator

In [ ]:
def visualized_creator_primarySecondary_Stats(wallet_address):
    creator_PrimarySecondary_Stats=numerical_creator_primarySecondary_Stats(wallet_address)
    creator_PrimarySecondary_Stats=plt.pie(creator_PrimarySecondary_Stats,shadow=True,labels=['At least one Sale on Secondary',
                                                        'Available on Primary','Available on Secondary'])